
# Exercise Notebook: Implementing RAG (Retrieval-Augmented Generation)

In this exercise notebook, you will go through the steps required to implement Retrieval-Augmented Generation (RAG).
The notebook will guide you through each step, providing explanations and asking you to fill in the code.

Please fill in the code cells where prompted to complete the implementation.

**Let's get started!**



## Installing Required Libraries

Before starting, ensure you have all the necessary libraries installed.
Install the following libraries by running the appropriate command below.

- `langchain`
- `langchain_community`
- `unstructured`
- `sentence_transformers`
- `tiktoken`
- `chromadb`
- `langchain_chroma`
- `langchain_groq`

Fill in the installation command in the code cell below:
Import Necessary Modules
Now, you need to import the necessary modules to build the RAG system. Write the import statements for the libraries required in the following code cell.

In [1]:
#instill libraries
%%capture
!pip install langchain
!pip install langchain_community
!pip install unstructured
!pip install sentence_transformers
!pip install tiktoken
!pip install chromadb
!pip install langchain_chroma
!pip install langchain_groq


## Import Necessary Modules

Now, you need to import the necessary modules to build the RAG system.
Write the import statements for the libraries required in the following code cell.


In [2]:
import os
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
import markdown
from langchain.text_splitter import RecursiveCharacterTextSplitter

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


# Data Pre-processing and Preparation

In this section, we will focus on preparing the dataset for retrieval-based models. The steps involve cleaning the text, tokenizing it, and vectorizing it for further use in our model. These steps are essential for efficient retrieval and generation.

In [3]:
!kaggle datasets download -d khaledzsa/example

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/example
License(s): unknown
 56% 6.00M/10.7M [00:00<00:00, 62.0MB/s]
100% 10.7M/10.7M [00:00<00:00, 61.0MB/s]


In [4]:
!unzip example.zip

Archive:  example.zip
  inflating: example.csv             


In [5]:
df = pd.read_csv('example.csv')
df.head()

,title,content
0,Geographic distribution and population,"According to the Indian census of 2001, there ..."
1,Language and literature,Malayalam is the language spoken by the Malaya...
2,Arrival of Cove Reber and Saosin EP (2004-2006),After the audition process and several guest v...
3,Formation and Translating the Name (2003-2004),"The original lineup for Saosin, consisting of ..."
4,Red Hot Organization and Tommy Boy Records dis...,"In 1996, Coolio appeared on the Red Hot Organi..."


In [6]:
import re

def clean_text(text):
    df['title']=df['title'].str.strip()
    df['content']=df['content'].str.strip()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+',' ', text).strip()
    return text

In [7]:
df.head()

,title,content
0,Geographic distribution and population,"According to the Indian census of 2001, there ..."
1,Language and literature,Malayalam is the language spoken by the Malaya...
2,Arrival of Cove Reber and Saosin EP (2004-2006),After the audition process and several guest v...
3,Formation and Translating the Name (2003-2004),"The original lineup for Saosin, consisting of ..."
4,Red Hot Organization and Tommy Boy Records dis...,"In 1996, Coolio appeared on the Red Hot Organi..."


In [8]:
#show dupplicait
df.duplicated().sum()

4724

In [9]:
#remove duplicates
df.drop_duplicates(inplace=True)

In [10]:
df.duplicated().sum()

0

In [11]:
#show mising value
df.isnull().sum()

,0
title,0
content,0


In [12]:
from nltk.tokenize import word_tokenize

def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorize_text(texts):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(texts)
    return vectors


# Read Files from the Directory

In this step, we will read all text-based files from a specified directory. The files could be in various formats such as Markdown (`.md`), plain text (`.txt`), or other similar formats. We will handle each file based on its extension and process it accordingly.

### Steps to Follow:

1. **Specify the directory**: Define the directory from which to load the files.
2. **Read files by extension**: Filter files based on their extensions (e.g., `.md`, `.txt`, etc.).
3. **Convert or process content**: For each file, load the content. For markdown files, we will convert them into HTML using the `markdown` module. For other text formats, we will simply read the content as plain text.
4. **Store the processed content**: The result of each file’s content will be stored in a list for further use.

In [14]:
directory = 'data/markdown_files'
os.makedirs(directory, exist_ok=True)

In [15]:
for i in range(0, 5_000):

    title = df['title'].iloc[i]
    content = df['content'].iloc[i]

    markdown_content = f"# {title}\n\n"
    markdown_content += f"{content}\n\n"

    with open(f'{directory}/{i}.md', 'w', encoding='utf-8') as file:
        file.write(markdown_content)

In [16]:
markdown_texts = []
for filename in os.listdir(directory):
  if filename.endswith(".md"):
    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
      markdown_content = file.read()
      html_content = markdown.markdown(markdown_content)
      markdown_texts.append(html_content)

## Split the Text into Chunks

In this step, we will split the text into manageable chunks. This is important for tasks such as document retrieval and text generation, where large bodies of text need to be broken down for efficient processing.

### Why Split Text into Chunks?

- **Memory Efficiency**: Working with smaller pieces of text is more memory efficient.
- **Improved Retrieval**: Splitting long documents into smaller sections can improve the relevance of retrieval tasks.
- **Better Generation**: For text generation, smaller chunks help models focus on a specific context.

### Steps to Follow:

1. **Specify the chunk size**: Define the maximum number of words or characters per chunk.
2. **Split the text**: Split each document or file content into chunks based on the defined size.
3. **Handle incomplete chunks**: If a document ends with a chunk that is smaller than the chunk size, include it as a valid chunk.
4. **Store the chunks**: Store all chunks in a list for further processing.

In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.create_documents(markdown_texts)

## Initialize the Embedding Model & Create a Vector Store Using Chroma

In this step, we will initialize an embedding model to convert text chunks into numerical vectors. These embeddings will be used to measure the similarity between different chunks of text. After generating the embeddings, we will store them using Chroma, a vector store designed to efficiently manage and retrieve embeddings.

### Steps to Follow:

1. **Initialize the embedding model**: Choose an embedding model (e.g., Sentence Transformers or OpenAI embeddings) to convert text into vectors.
2. **Generate embeddings**: Convert each text chunk into its corresponding embedding.
3. **Create a vector store**: Use Chroma to store the embeddings and their associated metadata (e.g., the original text chunk).
4. **Verify the store**: Ensure that the embeddings are stored correctly and that you can retrieve them based on similarity.

In [18]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(documents, embedding_function, persist_directory="./chroma_db")

<ipython-input-18-54f91d5a9530>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
import os
import json
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

# Load the Persistent Directory for Chroma DB

In this step, we will focus on **loading** the persistent storage for Chroma DB. This allows us to access previously stored embeddings and metadata without recomputing them. By setting up persistent storage, we ensure that the vector database can be saved to disk and loaded again when needed.

### Steps to Follow:

1. **Specify the persistent directory**: Identify the directory where the Chroma DB is stored.
2. **Load the vector store**: Use Chroma to load the embeddings and metadata from this directory.
3. **Verify the loaded data**: Ensure that the embeddings and associated data have been correctly loaded and can be queried.

In [20]:
PRESIST_DIRECTORY = '/content/chroma_db'
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
persist_directory = "./chroma_db"
db = Chroma(persist_directory=persist_directory, embedding_function=embedding_function)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
def query_chroma_db(query, db, top_k=5):
    docs = db.similarity_search(query)
    results = [doc.page_content for doc in docs]
    return results

In [22]:
query_chroma_db(" The number of Malayalam speakers in Lakshadweep", db)

['<p>According to the Indian census of 2001, there were 30,803,747 speakers of Malayalam in Kerala, making up 93.2% of the total number of Malayalam speakers in India, and 96.7% of the total population of the state. There were a further 701,673 (2.1% of the total number) in Karnataka, 557,705 (1.7%) in Tamil Nadu and 406,358 (1.2%) in Maharashtra. The number of Malayalam speakers in Lakshadweep is 51,100, which is only 0.15% of the total number, but is as much as about 84% of the population of',
 'but is as much as about 84% of the population of Lakshadweep. In all, Malayalis made up 3.22% of the total Indian population in 2001. Of the total 33,066,392 Malayalam speakers in India in 2001, 33,015,420 spoke the standard dialects, 19,643 spoke the Yerava dialect and 31,329 spoke non-standard regional variations like Eranadan. As per the 1991 census data, 28.85% of all Malayalam speakers in India spoke a second language and 19.64% of the total knew three or more languages.  Large numbers o

# Create & Test the Retrieval with a Sample Query

In this step, we will set up the retrieval process using the embeddings stored in Chroma DB. Retrieval is a key part of the Retrieval-Augmented Generation (RAG) pipeline, allowing us to find relevant documents or text chunks based on a query. After setting up the retrieval system, we will test it with a sample query to ensure that it returns the most relevant chunks.

### Steps to Follow:

1. **Set up the retrieval system**: Using the Chroma DB with the stored embeddings, create a retrieval function that can match a query to relevant text chunks.
2. **Prepare a sample query**: Define a query that you want to search for in the stored text chunks.
3. **Retrieve relevant chunks**: Use the query to search the vector store and retrieve the most similar chunks.
4. **Test the results**: Check that the returned chunks are relevant to the query and adjust the retrieval system if needed.

In [23]:
PROMPT_TEMPLATE="""
Answer the question based only on the following context:
Context: {context}
Question: {question}
Your answer:
"""

prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)

In [24]:
groq_api_key = "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"
llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-8b-8192")

In [25]:
MODEL = LLMChain(llm=llm,
                 prompt=prompt_template,
                 verbose=True)

<ipython-input-25-a2932d0d6a9b>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  MODEL = LLMChain(llm=llm,


In [26]:
def query_rag(query: str):
    similarity_search_results = db.similarity_search_with_score(query, k=4)
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search_results])

    rag_response = MODEL.invoke({"context": context_text, "question": query})

    return rag_response

In [27]:
response = query_rag("The number of Malayalam speakers in Lakshadweep")
response

Prompt after formatting:

Answer the question based only on the following context:
Context: <p>According to the Indian census of 2001, there were 30,803,747 speakers of Malayalam in Kerala, making up 93.2% of the total number of Malayalam speakers in India, and 96.7% of the total population of the state. There were a further 701,673 (2.1% of the total number) in Karnataka, 557,705 (1.7%) in Tamil Nadu and 406,358 (1.2%) in Maharashtra. The number of Malayalam speakers in Lakshadweep is 51,100, which is only 0.15% of the total number, but is as much as about 84% of the population of

but is as much as about 84% of the population of Lakshadweep. In all, Malayalis made up 3.22% of the total Indian population in 2001. Of the total 33,066,392 Malayalam speakers in India in 2001, 33,015,420 spoke the standard dialects, 19,643 spoke the Yerava dialect and 31,329 spoke non-standard regional variations like Eranadan. As per the 1991 census data, 28.85% of all Malayalam speakers in India spoke a

{'context': '<p>According to the Indian census of 2001, there were 30,803,747 speakers of Malayalam in Kerala, making up 93.2% of the total number of Malayalam speakers in India, and 96.7% of the total population of the state. There were a further 701,673 (2.1% of the total number) in Karnataka, 557,705 (1.7%) in Tamil Nadu and 406,358 (1.2%) in Maharashtra. The number of Malayalam speakers in Lakshadweep is 51,100, which is only 0.15% of the total number, but is as much as about 84% of the population of\n\nbut is as much as about 84% of the population of Lakshadweep. In all, Malayalis made up 3.22% of the total Indian population in 2001. Of the total 33,066,392 Malayalam speakers in India in 2001, 33,015,420 spoke the standard dialects, 19,643 spoke the Yerava dialect and 31,329 spoke non-standard regional variations like Eranadan. As per the 1991 census data, 28.85% of all Malayalam speakers in India spoke a second language and 19.64% of the total knew three or more languages.  Large

In [28]:
print(f'Context:\n{response["context"]}\n\nQuestion:\n{response["question"]}\n\nText: \n{response["text"]}')

Context:
<p>According to the Indian census of 2001, there were 30,803,747 speakers of Malayalam in Kerala, making up 93.2% of the total number of Malayalam speakers in India, and 96.7% of the total population of the state. There were a further 701,673 (2.1% of the total number) in Karnataka, 557,705 (1.7%) in Tamil Nadu and 406,358 (1.2%) in Maharashtra. The number of Malayalam speakers in Lakshadweep is 51,100, which is only 0.15% of the total number, but is as much as about 84% of the population of

but is as much as about 84% of the population of Lakshadweep. In all, Malayalis made up 3.22% of the total Indian population in 2001. Of the total 33,066,392 Malayalam speakers in India in 2001, 33,015,420 spoke the standard dialects, 19,643 spoke the Yerava dialect and 31,329 spoke non-standard regional variations like Eranadan. As per the 1991 census data, 28.85% of all Malayalam speakers in India spoke a second language and 19.64% of the total knew three or more languages.  Large numbe

In [29]:
query = "The number of Malayalam speakers in Lakshadweep"
similarity_search_results = db.similarity_search_with_score(query, k=4)

In [30]:
print("First: ", similarity_search_results[0][0].page_content)
print("Second: ", similarity_search_results[1][0].page_content)
print("Third: ", similarity_search_results[2][0].page_content)
print("Fourth: ", similarity_search_results[3][0].page_content)

First:  <p>According to the Indian census of 2001, there were 30,803,747 speakers of Malayalam in Kerala, making up 93.2% of the total number of Malayalam speakers in India, and 96.7% of the total population of the state. There were a further 701,673 (2.1% of the total number) in Karnataka, 557,705 (1.7%) in Tamil Nadu and 406,358 (1.2%) in Maharashtra. The number of Malayalam speakers in Lakshadweep is 51,100, which is only 0.15% of the total number, but is as much as about 84% of the population of
Second:  but is as much as about 84% of the population of Lakshadweep. In all, Malayalis made up 3.22% of the total Indian population in 2001. Of the total 33,066,392 Malayalam speakers in India in 2001, 33,015,420 spoke the standard dialects, 19,643 spoke the Yerava dialect and 31,329 spoke non-standard regional variations like Eranadan. As per the 1991 census data, 28.85% of all Malayalam speakers in India spoke a second language and 19.64% of the total knew three or more languages.  Larg

In [31]:
print(similarity_search_results[0][1])
print(similarity_search_results[1][1])
print(similarity_search_results[2][1])
print(similarity_search_results[3][1])

0.2781030535697937
0.4446018636226654
0.8611243963241577
0.8742256760597229


In [32]:
def query_rag_with_threshold(query: str, threshold: float):
    similarity_search_results = db.similarity_search_with_score(query, k=4)
    context_text = "\n\n".join([doc.page_content for doc, score in similarity_search_results if score > threshold])
    rag_response = MODEL.invoke({"context": context_text, "question": query})
    return rag_response

In [33]:
response = query_rag_with_threshold("The number of Malayalam speakers in Lakshadweep", 0.80)
response

Prompt after formatting:

Answer the question based only on the following context:
Context: 7,070 people who listed Malayalam as their mother tongue, mostly in the Greater Toronto Area and Southern Ontario. In 2010, the Census of Population of Singapore reported that there were 26,348 Malayalees in Singapore. The 2006 New Zealand census reported 2,139 speakers. 134 Malayalam speaking households were reported in 1956 in Fiji. There is also a considerable Malayali population in the Persian Gulf regions, especially in Bahrain, Muscat, Doha, Dubai, Abu Dhabi, Kuwait and European region

knew three or more languages.  Large numbers of Malayalis have settled in Bangalore, Mangalore, Delhi, Coimbatore, Hyderabad, Mumbai (Bombay), Ahmedabad, Pune, and Chennai (Madras). A large number of Malayalis have also emigrated to the Middle East, the United States, and Europe. Accessed November 22, 2014.</ref> including a large number of professionals. There were 7,093 Malayalam speakers in Australia in 

{'context': '7,070 people who listed Malayalam as their mother tongue, mostly in the Greater Toronto Area and Southern Ontario. In 2010, the Census of Population of Singapore reported that there were 26,348 Malayalees in Singapore. The 2006 New Zealand census reported 2,139 speakers. 134 Malayalam speaking households were reported in 1956 in Fiji. There is also a considerable Malayali population in the Persian Gulf regions, especially in Bahrain, Muscat, Doha, Dubai, Abu Dhabi, Kuwait and European region\n\nknew three or more languages.  Large numbers of Malayalis have settled in Bangalore, Mangalore, Delhi, Coimbatore, Hyderabad, Mumbai (Bombay), Ahmedabad, Pune, and Chennai (Madras). A large number of Malayalis have also emigrated to the Middle East, the United States, and Europe. Accessed November 22, 2014.</ref> including a large number of professionals. There were 7,093 Malayalam speakers in Australia in 2006. The 2001 Canadian census reported 7,070 people who listed Malayalam as 

In [36]:
print(f'Context:\n{response["context"]}\n\nQuestion:\n{response["question"]}\n\nText: \n{response["text"]}')

Context:
7,070 people who listed Malayalam as their mother tongue, mostly in the Greater Toronto Area and Southern Ontario. In 2010, the Census of Population of Singapore reported that there were 26,348 Malayalees in Singapore. The 2006 New Zealand census reported 2,139 speakers. 134 Malayalam speaking households were reported in 1956 in Fiji. There is also a considerable Malayali population in the Persian Gulf regions, especially in Bahrain, Muscat, Doha, Dubai, Abu Dhabi, Kuwait and European region

knew three or more languages.  Large numbers of Malayalis have settled in Bangalore, Mangalore, Delhi, Coimbatore, Hyderabad, Mumbai (Bombay), Ahmedabad, Pune, and Chennai (Madras). A large number of Malayalis have also emigrated to the Middle East, the United States, and Europe. Accessed November 22, 2014.</ref> including a large number of professionals. There were 7,093 Malayalam speakers in Australia in 2006. The 2001 Canadian census reported 7,070 people who listed Malayalam as their 